# This is supplementary matirial to the paper on [multiscale QTT]()

## Diffusion equation
First we consider the diffusion equation in one space dimension 
with homogeneous Dirichlet boundary conditions 
$$ 
\left( a u'\right)' = 1, \quad u(1) = u(0) = 0,
$$

$$a(x) = a_0(x) a_1\left(\frac{x}{\delta}\right),$$

where $a_0(x) = 1 + x$, $a_1(y) = \frac{2}{3} \left(1 + \cos^2\left(2 \pi y\right)\right)$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.sparse.linalg
import scipy as sp
import scipy.interpolate
import math
import numpy as np
import tt

In [ ]:
def Kfun(x, eps):
    return (1 + np.sqrt(x)) * 2/3 * (1 + np.cos(2 * math.pi * (x/eps)) ** 2)

def rhs_fun(x):
    return np.ones(x.shape)

def derivative(arr, fun, h):
    N = arr.shape[0]
    res = np.zeros(N)
    res[1:-1] = (arr[2:] - arr[:-2])/(2*h)
    res[0] = (arr[1] - 0.)/(2*h)
    res[-1] = (0. - arr[-2])/(2*h)
    mesh = np.linspace(0, 1, N + 2)[1:-1]
    return res #* fun(mesh)

def gen_full_matrix(d, eps, Kfun, a=0.0, b=1.0): 
    N = 2 ** d - 1
    t = np.arange(N)
    e = np.ones(N)
    h = (b - a)/(N + 1)
    cf = (t + 0.5 * e) * h
    am = Kfun(cf, eps)
    cf_plus = (t + 1.5 * e) * h
    ap = Kfun(cf_plus, eps)
    dg = am + ap
    dg = np.concatenate(([h**2], dg))
    am = np.concatenate((am, [0]))
    ap = np.concatenate(([0, 0], ap[:-1]))
    mat = scipy.sparse.spdiags([-am, -ap, dg], [-1, 1, 0], N+1, N+1, format='csr')
    mat = mat/h**2
    return mat

### Massive experiments

In [ ]:
dall = [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
eps_all = 10 ** (-np.linspace(1, 5, 53)) #[1e-0, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
tau_all = 10 ** (-np.linspace(1, 11, 101))

res = {}
for eps in eps_all:
    print eps
    dims = []
    sols = []
    sols_interp = []
    h_all = []
    meshes = []
    matrs = []

    for d in dall:
        i = d - dall[0]
        dims.append(d)
        N = 2 ** d
        h = 1.0/(N)
        h_all.append(h)
        mat = gen_full_matrix(d, eps, Kfun, a=0.0, b=1.0)
        meshes.append(np.linspace(0, 1, N+1))
        rhs = rhs_fun(meshes[i][:-1])
        result = np.zeros(N + 1)
        result[:-1] = sp.sparse.linalg.spsolve(mat, rhs)
        sols.append(result)

    for d in dall[:-2]:
        i = d - dall[0]
        h = h_all[i+2]
        solh = sols[i+2]
        sol2h = np.zeros(sols[i+2].shape)
        sol2h[::2] = sols[i+1]
        sol2h[1::2] = (sols[i+1][:-1] + sols[i+1][1:])/2
        sol_extrap = solh + (solh - sol2h) / (2.0**2 + 1)
        kh = (sols[i+2][1:] - sols[i+2][:-1]) / h
        kh_extrap = (sol_extrap[1:] - sol_extrap[:-1])/h
        
        for tau in tau_all:
            solh_qtt_full = np.zeros(len(solh))
            solh_qtt = tt.tensor(solh[:-1].reshape(2*np.ones(d+2), order='F'), tau)
            solh_qtt_full[:-1] = solh_qtt.full().flatten('F')
            kh_qtt_full = (solh_qtt_full[1:] - solh_qtt_full[:-1]) / h
            res[(d+2, tau, eps, 'memory')] = len(solh_qtt.core)
            res[(d+2, tau, eps, 'h1_qtt')] = np.linalg.norm(kh_qtt_full - kh_extrap) * np.sqrt(h)
            res[(d+2, tau, eps, 'h1')] = np.linalg.norm(kh - kh_extrap) * np.sqrt(h) 
            res[(d+2, tau, eps, 'l2')] = np.linalg.norm(solh - sol_extrap) * np.sqrt(h) 
                                   
            
np.savez('computations_diffusion', res=res, dall=dall[2:], eps_all=eps_all, tau_all=tau_all)

## Figures

Figures can be found [here](plots_diffusion.ipynb).

## 1D Helmholtz 

As the second example, we consider  the one-dimensional Helmholtz problem with radiation boundary conditions:
$$-u''  - k^2 u = 1, \quad u(0) = 0, \quad u'(1) - i k u(1) = 0.$$

In [ ]:
#Generate the tridiagonal sparse matrix
import numpy as np
import scipy.linalg
from scipy import interpolate

def gen_helmholtz_matrix(d, K, a=0.0, b=1.0):
    N = 2 ** d
    t = np.linspace(a, b, N+1) #Left point ommited
    h = 1. / N
    ap = -np.ones(N) / h ** 2 - K**2 / 6
    am = -np.ones(N) / h ** 2 - K**2 / 6
    dg = 2 * np.ones(N, dtype=np.complex128)/h**2 - (K**2)*2./3
    dg[0] = 2./h**2 - (K**2) * 1./3 
    dg[N-1] = 1.0 / h**2 - (K**2)/3 - 1j*K / h  #Radiation b.c.
    mat = scipy.sparse.spdiags([am, ap, dg], [1, -1, 0], N, N).tocsc()
    return mat


def rhs_fun(x):
    res = np.zeros(x.shape, dtype=np.complex128)
    res[0] = 1.
    return res #np.ones(x.shape, dtype=np.complex128)

def solution_fun(x, k):
    return (1j/k) * (1.-np.exp(1j*k)/2) * np.exp(1j*k*x) - 1j*(np.exp(1j*k)/(2*k)) * np.exp(-1j*k*x)

def trig_squared_integral(c1, c2, c3, x, k): #integrates (c1 cos kx + c2 sin kx + c3)^2
    return c1**2 * (x/2 + np.sin(2*k*x)/(4*k)) + c2**2 * (x/2 - np.sin(2*k*x)/(4*k)) + c3**2*x - \
           c1*c2*np.cos(2*k*x)/(2*k) + 2*c1*c3*np.sin(k*x)/k - 2*c2*c3*np.cos(k*x)/k 
    
def h1_norm_func2(kh, h, k, N):
    c1 = 1.0j/k * (1 - np.exp(1j*k)/2)
    c2 = -1.0j*np.exp(1j*k)/(2*k)
    res = trig_squared_integral(c1.real+c2.real, -c1.imag+c2.imag, -kh.real, h*np.arange(1, N), k) - \
          trig_squared_integral(c1.real+c2.real, -c1.imag+c2.imag, -kh.real, h*np.arange(0, N-1), k) + \
          trig_squared_integral(c1.imag+c2.imag, c1.real-c2.real, -kh.imag, h*np.arange(1, N), k) - \
          trig_squared_integral(c1.imag+c2.imag, c1.real-c2.real, -kh.imag, h*np.arange(0, N-1), k)
    return np.sqrt(res.sum())

### Massive experiments

In [ ]:
dall = [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
K_all = [1., 5., 10., 50., 100., 500.,  1000., 5000., 10000.]
tau_all = 10 ** (-np.linspace(1, 11, 101))

res = {}
for K in K_all:
    print 'k =', K
    dims = []
    sols = []
    sols_interp = []
    h_all = []
    meshes = []
    matrs = []

    for d in dall:
        i = d - dall[0]
        dims.append(d)
        N = 2 ** d
        h = 1.0/(N)
        h_all.append(h)
        mat = gen_helmholtz_matrix(d, K, a=0.0, b=1.0)
        meshes.append(np.linspace(0, 1, N+1))
        rhs = np.ones(N, dtype=np.complex128)
        result = np.zeros(N+1, dtype=np.complex128)
        result[1:] = sp.sparse.linalg.spsolve(mat, rhs)
        sols.append(result)

    for d in dall[:-2]:
        i = d - dall[0]
        h = h_all[i+2]   
        solh = sols[i+2]
        sol2h = np.zeros(sols[i+2].shape, dtype=np.complex128)
        sol2h[::2] = sols[i+1]
        sol2h[1::2] = (sols[i+1][:-1] + sols[i+1][1:])/2
        kh = (sols[i+2][1:] - sols[i+2][:-1]) / h
        
        for tau in tau_all:
            solh_qtt_full = np.zeros(len(solh), dtype=np.complex128)
            solh_qtt = tt.tensor(solh[1:].reshape(2*np.ones(d+2), order='F'), tau)
            solh_qtt_full[1:] = solh_qtt.full().flatten('F')
            kh_qtt_full = (solh_qtt_full[1:] - solh_qtt_full[:-1]) / h
            res[(d+2, tau, K, 'memory')] = len(solh_qtt.core)
            res[(d+2, tau, K, 'h1_qtt')] = h1_norm_func2(kh_qtt_full, h, K, 2**(d+2)+1)
            res[(d+2, tau, K, 'h1')] = h1_norm_func2(kh, h, K, 2**(d+2)+1)
            res[(d+2, tau, K, 'l2')] = np.linalg.norm(solh - sol_extrap) * np.sqrt(h) 
            res[(d+2, tau, K, 'erank')] = solh_qtt.erank
                                   
            
np.savez('computations_helmholtz', res=res, dall=dall[2:], K_all=K_all, tau_all=tau_all)

## Figures

Figures can be found [here](plots_helmholtz.ipynb).